# Genetic optimisation of SVC classifier for the SDSS data

## Load data

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import warnings
import helpers
from helpers import DataSet
import matplotlib as mpl
import os

# Common imports
import pandas as pd
import numpy as np
import seaborn as sns

# Imports for ML
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
sns.set_style('whitegrid')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "results"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# Helper functioins and structures
# Ignore useless warnings (see SciPy issue #5998)
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATA_PATH = "Skyserver_SQL2_27_2018 6_51_39 PM.csv"
RESULTS_FOLDER = "results"

# We load the data. Those that have nothing to do with the features of the objects are ignored.
sdss_data = pd.read_csv(DATA_PATH)

# We have a general look at the features
sdss_data.head(3)

print(sdss_data.columns.values)
sdss_data.drop(['objid', 'run', 'rerun', 'camcol', 'field',
                'specobjid', 'fiberid', 'mjd', 'plate'], axis=1, inplace=True)
sdss_data.head(1)

sdss_df_fe = sdss_data

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

X = sdss_data.drop(['class'], axis=1)
y = sdss_data['class']

class_names = ["GALAXY", "QSO", "STAR"]

std_scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_test = std_scaler.fit_transform(X_test.astype(np.float64))
X_train = std_scaler.fit_transform(X_train.astype(np.float64))

['objid' 'ra' 'dec' 'u' 'g' 'r' 'i' 'z' 'run' 'rerun' 'camcol' 'field'
 'specobjid' 'class' 'redshift' 'plate' 'mjd' 'fiberid']


## Optimize

In [2]:
import random


from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

paramgrid = {"kernel": ["rbf"],
             "C": np.logspace(-9, 5, num=20, base=10),
             "gamma": np.logspace(-9, 5, num=20, base=10),
             "degree": range(3,10),
             "coef0": np.logspace(-9, 5, num=20, base=10),
             "shrinking": [True, False],
             "probability": [False, True],
             "tol": [0.001],
             "cache_size": [20, 50, 100, 200, 500, 1000],
             "class_weight": [None],
             "verbose": [False],
             "max_iter": [-1],
             "random_state": [42]}

random.seed(1)

from evolutionary_search import EvolutionaryAlgorithmSearchCV
cv = EvolutionaryAlgorithmSearchCV(estimator=SVC(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=5),
                                   verbose=3,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=15,
                                   n_jobs=8,
                                   error_score=0.0)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = cv.fit(X_train, y_train)

Types [1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1] and maxint [0, 19, 19, 6, 19, 1, 1, 0, 5, 0, 0, 0, 0] detected
[CV] kernel=rbf, C=3359.8182862837743, gamma=1e-09, degree=6, coef0=2.6366508987303556e-05, shrinking=False, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV] kernel=rbf, C=18329.8071083243, gamma=112.88378916846884, degree=9, coef0=615.8482110660254, shrinking=False, probability=False, tol=0.001, cache_size=50, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV] kernel=rbf, C=18329.8071083243, gamma=2.976351441631319e-08, degree=5, coef0=1.623776739188721e-07, shrinking=False, probability=True, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV] kernel=rbf, C=100000.0, gamma=18329.8071083243, degree=7, coef0=0.6951927961775591, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[C

[CV]  kernel=rbf, C=20.6913808111479, gamma=0.004281332398719387, degree=3, coef0=3.7926901907322383, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9700199866755497, total=   0.7s
[CV] kernel=rbf, C=20.6913808111479, gamma=0.004281332398719387, degree=3, coef0=3.7926901907322383, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=0.004281332398719387, gamma=8.858667904100833e-07, degree=4, coef0=5.455594781168515e-09, shrinking=False, probability=False, tol=0.001, cache_size=100, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.5003335557038026, total=   2.0s
[CV] kernel=rbf, C=1e-09, gamma=3359.8182862837743, degree=9, coef0=5.455594781168515e-09, shrinking=True, probability=True, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=

[CV]  kernel=rbf, C=18329.8071083243, gamma=2.976351441631319e-08, degree=5, coef0=1.623776739188721e-07, shrinking=False, probability=True, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.7951967978652434, total=   5.5s
[CV] kernel=rbf, C=18329.8071083243, gamma=2.976351441631319e-08, degree=5, coef0=1.623776739188721e-07, shrinking=False, probability=True, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=18329.8071083243, gamma=112.88378916846884, degree=9, coef0=615.8482110660254, shrinking=False, probability=False, tol=0.001, cache_size=50, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.6506666666666666, total=   8.7s
[CV] kernel=rbf, C=18329.8071083243, gamma=112.88378916846884, degree=9, coef0=615.8482110660254, shrinking=False, probability=False, tol=0.001, cache_size=50, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  

[CV]  kernel=rbf, C=1e-09, gamma=3359.8182862837743, degree=9, coef0=5.455594781168515e-09, shrinking=True, probability=True, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.5, total=  12.6s
[CV] kernel=rbf, C=1e-09, gamma=3359.8182862837743, degree=9, coef0=5.455594781168515e-09, shrinking=True, probability=True, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=2.976351441631319e-08, gamma=20.6913808111479, degree=8, coef0=615.8482110660254, shrinking=True, probability=False, tol=0.001, cache_size=100, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.5003331112591606, total=   2.8s
[CV] kernel=rbf, C=2.976351441631319e-08, gamma=20.6913808111479, degree=8, coef0=615.8482110660254, shrinking=True, probability=False, tol=0.001, cache_size=100, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=2.976351441631319e-08, ga

[CV]  kernel=rbf, C=0.004281332398719387, gamma=20.6913808111479, degree=8, coef0=0.023357214690901212, shrinking=False, probability=True, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.5003331112591606, total=  12.6s
[CV] kernel=rbf, C=0.004281332398719387, gamma=20.6913808111479, degree=8, coef0=0.023357214690901212, shrinking=False, probability=True, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=0.6951927961775591, gamma=3.7926901907322383, degree=7, coef0=1e-09, shrinking=False, probability=True, tol=0.001, cache_size=500, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9266177451634423, total=   8.0s
[CV] kernel=rbf, C=18329.8071083243, gamma=3359.8182862837743, degree=9, coef0=1.623776739188721e-07, shrinking=False, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=

...

[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=1.623776739188721e-07, shrinking=True, probability=False, tol=0.001, cache_size=100, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=1.623776739188721e-07, gamma=0.004281332398719387, degree=5, coef0=112.88378916846884, shrinking=True, probability=True, tol=0.001, cache_size=100, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.5003335557038026, total=   7.1s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=3.7926901907322383, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=2.6366508987303556e-05, shrinking=True, probability=False, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.992, total=   2.4s
[CV] kernel=rbf, C=100000.0, gamma=

[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=615.8482110660254, shrinking=True, probability=False, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9913391072618255, total=   2.8s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=615.8482110660254, shrinking=True, probability=False, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=6, coef0=2.6366508987303556e-05, shrinking=True, probability=True, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9913391072618255, total=  15.2s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=6, coef0=2.6366508987303556e-05, shrinking=True, probability=True, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.

[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=0.004281332398719387, shrinking=True, probability=True, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=8.858667904100833e-07, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9926617745163442, total=   3.5s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=8.858667904100833e-07, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=0.0001438449888287663, shrinking=True, probability=True, tol=0.001, cache_size=100, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9926617745163442, total=  15.6s
[CV] kernel=rbf, C=100000.0, gamma=0

[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=615.8482110660254, shrinking=True, probability=False, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9913391072618255, total=   3.4s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=615.8482110660254, shrinking=True, probability=False, tol=0.001, cache_size=1000, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=1.623776739188721e-07, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9866755496335776, total=   2.5s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=1.623776739188721e-07, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=

[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=1.623776739188721e-07, shrinking=True, probability=True, tol=0.001, cache_size=500, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9926617745163442, total=  10.0s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=4, coef0=1.623776739188721e-07, shrinking=True, probability=True, tol=0.001, cache_size=500, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=4.832930238571752e-06, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42, score=0.9913391072618255, total=   3.2s
[CV] kernel=rbf, C=100000.0, gamma=0.004281332398719387, degree=8, coef0=4.832930238571752e-06, shrinking=True, probability=False, tol=0.001, cache_size=20, class_weight=None, verbose=False, max_iter=-1, random_state=42 
[CV]  kernel=rbf, C=100000.0, gamm